External datapoints to leverage:
- Vacations
- Weather
- Lockdowns / curfews
- Fuel prices
- 

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

data = pd.read_parquet(Path('data') / 'train.parquet')
data.groupby(['site_name', 'counter_name'])['bike_count'].sum().sort_values(ascending=False).head(10).to_frame()

In [16]:
df_ext = pd.read_csv(Path('submissions') / 'starting_kit' / 'external_data.csv')
df_ext.head()

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,hnuage1,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4
0,7149,2021-01-01 00:00:00,100810,80,1,270,1.8,272.75,272.15,96,...,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7149,2021-01-01 03:00:00,100920,110,3,300,1.7,271.25,270.95,98,...,1500.0,2.0,3.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN
2,7149,2021-01-01 06:00:00,100950,30,3,290,2.6,271.95,271.65,98,...,480.0,4.0,6.0,2000.0,6.0,3.0,3000.0,NaN,NaN,NaN
3,7149,2021-01-01 09:00:00,101100,150,2,280,1.7,272.45,272.05,97,...,1740.0,3.0,3.0,2800.0,NaN,NaN,NaN,NaN,NaN,NaN
4,7149,2021-01-01 12:00:00,101110,30,0,50,1.0,276.95,274.15,82,...,330.0,4.0,6.0,570.0,7.0,6.0,810.0,NaN,NaN,NaN


In [14]:
data.loc[:, 'year'] = data['date'].dt.year
data.loc[:, 'month'] = data['date'].dt.month
data.loc[:, 'day'] = data['date'].dt.day
data.loc[:, 'weekday'] = data['date'].dt.weekday
data.loc[:, 'hour'] = data['date'].dt.hour
data_train = data[['counter_name', 'bike_count', 'log_bike_count', 'year', 'month', 'day', 'weekday', 'hour']]

In [15]:
data_train

,counter_name,bike_count,log_bike_count,year,month,day,weekday,hour
48321,28 boulevard Diderot E-O,0.0,0.000000,2020,9,1,1,2
48324,28 boulevard Diderot E-O,1.0,0.693147,2020,9,1,1,3
48327,28 boulevard Diderot E-O,0.0,0.000000,2020,9,1,1,4
48330,28 boulevard Diderot E-O,4.0,1.609438,2020,9,1,1,15
48333,28 boulevard Diderot E-O,9.0,2.302585,2020,9,1,1,18
...,...,...,...,...,...,...,...,...
928450,254 rue de Vaugirard SO-NE,51.0,3.951244,2021,8,8,6,18
928453,254 rue de Vaugirard SO-NE,1.0,0.693147,2021,8,9,0,2
928456,254 rue de Vaugirard SO-NE,61.0,4.127134,2021,8,9,0,8
928459,254 rue de Vaugirard SO-NE,44.0,3.806662,2021,8,9,0,10
